# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [5]:
import pandas as pd
import numpy as np
import os

## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [12]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  78182      0 --:--:-- --:--:-- --:--:-- 78184
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   2553      0 --:--:-- --:--:-- --:--:--  2553
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   2720      0 --:--:-- --:--:-- --:--:--  2725


In [13]:
!ls

03.06-Concat-And-Append.ipynb  data		      data_exploration.ipynb
03.07-Merge-and-Join.ipynb     Data Cleaning 1.ipynb


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [6]:
pwd = os.getcwd()

In [7]:
#Modified according to windows machine
pop = pd.read_csv(pwd+'/data_cleaning_caste_study/state-population.csv')
areas = pd.read_csv(pwd+'/data_cleaning_caste_study/state-areas.csv')
abbrevs = pd.read_csv(pwd+'/data_cleaning_caste_study/state-abbrevs.csv')



Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

In [18]:
pop.head(1)

,state/region,ages,year,population,tmp
0,AL,under18,2012,1117489.0,1


In [20]:
pop.shape

(2544, 5)

In [21]:
abbrevs.head(1)

,state,abbreviation,tmp
0,Alabama,AL,1


In [22]:
abbrevs.shape

(51, 3)

In [23]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

,state/region,ages,year,population,tmp_x,state,tmp_y
0,AL,under18,2012,1117489.0,1,Alabama,1.0
1,AL,total,2012,4817528.0,1,Alabama,1.0
2,AL,under18,2010,1130966.0,1,Alabama,1.0
3,AL,total,2010,4785570.0,1,Alabama,1.0
4,AL,under18,2011,1125763.0,1,Alabama,1.0


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [24]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
tmp_x           False
state            True
tmp_y            True
dtype: bool

## Look for any rows with null values in the new dataframe

In [25]:
merged.isnull().any(axis=1)

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2514     True
2515     True
2516     True
2517     True
2518     True
2519     True
2520     True
2521     True
2522     True
2523     True
2524     True
2525     True
2526     True
2527     True
2528     True
2529     True
2530     True
2531     True
2532     True
2533     True
2534     True
2535     True
2536     True
2537     True
2538     True
2539     True
2540     True
2541     True
2542     True
2543     True
Length: 2544, dtype: bool

## Print out the rows in the merged dataframe that has null values for population.

In [26]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,tmp_x,state,tmp_y
2448,PR,under18,1990,NaN,1,NaN,NaN
2449,PR,total,1990,NaN,1,NaN,NaN
2450,PR,total,1991,NaN,1,NaN,NaN
2451,PR,under18,1991,NaN,1,NaN,NaN
2452,PR,total,1993,NaN,1,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [27]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [28]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
tmp_x           False
state           False
tmp_y            True
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [29]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,tmp_x,state,tmp_y,area (sq. mi)
0,AL,under18,2012,1117489.0,1,Alabama,1.0,52423.0
1,AL,total,2012,4817528.0,1,Alabama,1.0,52423.0
2,AL,under18,2010,1130966.0,1,Alabama,1.0,52423.0
3,AL,total,2010,4785570.0,1,Alabama,1.0,52423.0
4,AL,under18,2011,1125763.0,1,Alabama,1.0,52423.0


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [30]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
tmp_x            False
state            False
tmp_y             True
area (sq. mi)     True
dtype: bool

## Select the portion of the data corresponding with the year 2000, and the total population.

In [34]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,tmp_x,state,tmp_y,area (sq. mi)
3,AL,total,2010,4785570.0,1,Alabama,1.0,52423.0
91,AK,total,2010,713868.0,1,Alaska,1.0,656425.0
101,AZ,total,2010,6408790.0,1,Arizona,1.0,114006.0
189,AR,total,2010,2922280.0,1,Arkansas,1.0,53182.0
197,CA,total,2010,37333601.0,1,California,1.0,163707.0


## Re-index the data on state and find population density and display by density in ascending order

In [35]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [36]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

In [37]:
density.tail()

state
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
United States         NaN
dtype: float64

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!